<a href="https://colab.research.google.com/github/rtajeong/M3_new/blob/main/M3_lab51_bag_of_word.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bag of Word (BOW)
- Document-Term Matrix and Tfidf Matrix

뉴스기사 분석
- 문서-단어 행렬의 예시 (Document-term Matrix)
- 빅카인즈 뉴스기사 데이터 활용(https://www.bigkinds.or.kr/): 한국언론진흥재단이 운영하는 사이트로
  국내 언론사들 뉴스기사의 분야별 키워드, 통계, 이슈 등을 제공한다.
- 여기서는 2019년 뉴스 데이터 사용

In [1]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [2]:
# 뉴스 데이터 가져오기
!curl -L https://bit.ly/2X7UON2 -o news.xlsx

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   162  100   162    0     0   1820      0 --:--:-- --:--:-- --:--:--  1820
100   158    0   158    0     0    560      0 --:--:-- --:--:-- --:--:--  1112
100   340  100   340    0     0    559      0 --:--:-- --:--:-- --:--:--   559
100   534    0   534    0     0    680      0 --:--:-- --:--:-- --:--:--   680
100 1789k  100 1789k    0     0  1625k      0  0:00:01  0:00:01 --:--:-- 1625k


In [3]:
# !dir

In [4]:
news_all = pd.read_excel("news.xlsx")
news_all.columns

/usr/local/lib/python3.7/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Index(['뉴스 식별자', '일자', '언론사', '기고자', '제목', '통합 분류1', '통합 분류2', '통합 분류3',
       '사건/사고 분류1', '사건/사고 분류2', '사건/사고 분류3', '인물', '위치', '기관', '키워드', '특성추출',
       '본문', 'URL', '분석제외 여부'],
      dtype='object')

In [5]:
news_all.head(2).T

,0,1
뉴스 식별자,1500401.201902,1200201.201902
일자,20190228,20190228
언론사,국제신문,경인일보
기고자,김태경 기자 tgkim@kookje.co.kr,조영상
제목,빅딜이냐 스몰딜이냐 ‘영변 핵+α’ 로드맵 도출이 관건,"하노이 도착후 실무대표단 청취 김정은, 북미회담 성공 '잰걸음'"
통합 분류1,정치>북한,국제>국제일반
통합 분류2,정치>외교,정치>북한
통합 분류3,NaN,NaN
사건/사고 분류1,NaN,NaN
사건/사고 분류2,NaN,NaN


In [6]:
news_text = news_all['본문']
news_text[:5]

0    - 비핵화 수준 상응 조치 놓고\n- 양국 협상팀 막판까지 ‘밀당’\n- 1차 때와...
1    김정은 국무위원장이 27일 시작되는 제2차 북미정상회담 성공을 위해 심혈을 기울이고...
2    북미가 처음으로 정상 간 단독회담과 만찬을 가지며 또다시 새로운 역사 창조에 나섰다...
3    지난해 9월 남북정상회담 당시 리선권 북한 조국평화통일위원장의 '냉면' 발언으로 정...
4    지자체 민간 교류 활성화 대부분 \n여, 부처간 논의 예산 지원 확대 \n야, 사업...
Name: 본문, dtype: object

In [7]:
len(news_text[0])

200

In [8]:
type(news_text)

pandas.core.series.Series

In [9]:
cv = CountVectorizer()
dtm = cv.fit_transform(news_text)
df = pd.DataFrame(pd.DataFrame(dtm.toarray(), columns = cv.get_feature_names()))
df[:5]

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,00,000원을,001420,001550,002100,005690,01,017800,02,025860,...,힌국당,힘겨루기로,힘겨웠던,힘들다고,힘들어,힘들어지는,힘을,힘이,힘입어,靈山
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
df.shape    # 총 단어의 수가 16,199 개

(1543, 16199)

In [11]:
# 어떤 단어들이 등장했는지 확인 - dict
len(cv.vocabulary_.items())

16199

In [12]:
# 문서-단어 행렬에서 가장 많이 등장하는 단어 찾기 (이것이 훨씬 간단)
df.sum()    # column-wise sum
pd.Series(df.sum()).idxmax()

'2차'

In [13]:
# DSAC 교재에 나타난 코드 (결과는 위와 동일)
top_word = []
for i in df.T.values:    # df.T.values 와 동일 (2-d array)
    top_word.append(sum(i.tolist()))
    
df.transpose().index[top_word.index(max(top_word))]

'2차'

In [14]:
# maximum 200 개만 사용하도록 하기 위해서는?
cv = CountVectorizer(max_features=200)
dtm = cv.fit_transform(news_text)
df = pd.DataFrame(pd.DataFrame(dtm.toarray(), columns = cv.get_feature_names()))
print(df.shape)
df[:6]

(1543, 200)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,10일,11일,17일,18일,19일,1차,21일,24일,25일,26일,...,함께,했다,헤럴드경제,현지시간,협상,협상을,홍영표,회담,회담을,회담이
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
df.max(axis=1)

0       2
1       2
2       1
3       1
4       1
       ..
1538    2
1539    2
1540    1
1541    2
1542    2
Length: 1543, dtype: int64

In [16]:
tv = TfidfVectorizer(max_features=200)
dtm = tv.fit_transform(news_text.values)
df = pd.DataFrame(pd.DataFrame(dtm.toarray(), columns = tv.get_feature_names()))
print(df.shape)
df[:6]

(1543, 200)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,10일,11일,17일,18일,19일,1차,21일,24일,25일,26일,...,함께,했다,헤럴드경제,현지시간,협상,협상을,홍영표,회담,회담을,회담이
0,0.0,0.0,0.0,0.0,0.0,0.346577,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.287727,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.258984,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.274912,0.0,0.0,0.0,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0


# 연습

In [17]:
df2 = pd.DataFrame(np.arange(12).reshape(3,4)); df2
df2.values

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [18]:
df2.sum()

0    12
1    15
2    18
3    21
dtype: int64

In [19]:
df2.max()

0     8
1     9
2    10
3    11
dtype: int64

In [20]:
df2.idxmax(axis=1)

0    3
1    3
2    3
dtype: int64

In [21]:
df2.T.idxmax()

0    3
1    3
2    3
dtype: int64

In [22]:
for i in df2.values:
    print(sum(i))

6
22
38


In [23]:
tmp=[]
for i in df2.T.values:
    tmp.append(sum(i))

In [24]:
tmp

[12, 15, 18, 21]

In [25]:
tmp=np.array(df2.sum()); tmp

array([12, 15, 18, 21])

In [26]:
tmp_s = pd.Series(tmp)
tmp_s.idxmax()

3